<a href="https://colab.research.google.com/github/pragneshbarik/otaku-source/blob/master/otaku-model/otaku_recommender_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pragneshbarik","key":"f9acc8ebfa379a98564142560af8dfb9"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d marlesson/myanimelist-dataset-animes-profiles-reviews
! mkdir anime_dataset
! unzip /content/myanimelist-dataset-animes-profiles-reviews.zip -d anime_dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 93% 202M/217M [00:01<00:00, 159MB/s]
100% 217M/217M [00:01<00:00, 124MB/s]
mkdir: cannot create directory ‘anime_dataset’: File exists
Archive:  /content/myanimelist-dataset-animes-profiles-reviews.zip
  inflating: anime_dataset/animes.csv  
  inflating: anime_dataset/profiles.csv  
  inflating: anime_dataset/reviews.csv  


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def punctuation_replacer(x) :
  to_replace = [':', "!", "?", "-", '"', ";", "+", "\r", "\n"]
  for c in to_replace :
    x = x.replace(c, "")
  
  return x

ps = PorterStemmer()
def stemmer(text) :
  stemmed_text = []
  for word in text.split() :
    stemmed_text.append(ps.stem(word))
  
  return " ".join(stemmed_text)

def recommend(uid) :
  idx = anime_df[anime_df['uid']==uid].index[0]
  args = (-similarity_matrix[idx]).argsort()[:100]
  return args

In [ ]:
anime_df = pd.read_csv("/content/anime_dataset/animes.csv")
anime_df = anime_df.drop_duplicates(subset=['uid']).dropna(subset=['synopsis', 'img_url']).fillna(0)
anime_df['id']=np.arange(len(anime_df['uid']))
anime_df.set_index('id', drop=True, inplace=True)

In [ ]:
# anime_df['synopsis']=anime_df['synopsis'].apply(punctuation_replacer).apply(lambda x:re.split(', |"|,|-| ' , x))
# anime_df['genre']=anime_df['genre'].apply(lambda x: x.replace("'", "")).apply(lambda x:x[1:-1].split(',')).apply(lambda x:[i.replace(" ", '') for i in x])
anime_df['genre']=anime_df['genre'].apply(lambda x: x.replace("'", "")).apply(lambda x:x[1:-1].split(','))
anime_df['title_mod']  = anime_df['title'].apply(punctuation_replacer).apply(lambda x:x.split(" "))

In [ ]:
anime_df['tags'] = anime_df['genre']+anime_df['synopsis']
tagged_anime_df = anime_df[['uid', 'title', 'tags']]
tagged_anime_df['tags']=tagged_anime_df['tags'].apply(lambda x:" ".join(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
vectorizer = CountVectorizer(max_features=5000, stop_words='english')
anime_vectors = vectorizer.fit_transform(tagged_anime_df['tags']).toarray()
tagged_anime_df['tags'] = tagged_anime_df['tags'].apply(stemmer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
similarity_matrix = cosine_similarity(anime_vectors)

In [ ]:
picks = pd.DataFrame()
picks["id"] = anime_df.index
picks['uid'] = anime_df['uid']
picks['title'] = anime_df['title']
picks['recommendations']= anime_df['uid'].apply(recommend)

In [ ]:
picks.to_json('otaku-anime-picks.json', orient="index")

In [ ]:
picks

In [ ]:
picks

In [ ]:
anime_cache = anime_df.drop(columns=['title_mod', 'aired', 'episodes', 'link'])

In [ ]:
anime_cache.to_json('otaku-anime-database.json', orient='index')

In [ ]:
anime_cache

,uid,title,synopsis,genre,members,popularity,ranked,score,img_url
id,,,,,,,,,
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"[Comedy, Sports, Drama, School, Shounen]",489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"[Drama, Music, Romance, School, Shounen]",995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"[Sci-Fi, Adventure, Mystery, Drama, Fantasy]",581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","[Action, Military, Adventure, Comedy, Dram...",1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"[Action, Mystery, Supernatural, Vampire]",214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...
...,...,...,...,...,...,...,...,...,...
15307,10075,Naruto x UT,All-new animation offered throughout UNIQLO cl...,"[Action, Comedy, Super Power, Martial Arts,...",34155,2382,1728.0,7.50,https://cdn.myanimelist.net/images/anime/3/304...
15308,35828,Miira no Kaikata,High school student Sora Kashiwagi is accustom...,"[Slice of Life, Comedy, Supernatural]",61459,1648,1727.0,7.50,https://cdn.myanimelist.net/images/anime/1486/...
15309,10378,Shinryaku!? Ika Musume,"After regaining her squid-like abilities, Ika ...","[Slice of Life, Comedy, Shounen]",67422,1547,1548.0,7.56,https://cdn.myanimelist.net/images/anime/6/301...
